## GoF tests

In [33]:
import pandas as pd
import numpy as np
data = np.array([[50000,60000,100000,35000],[250,300,600,150]])
columns = ["Dark", "Milk", "White", "Other"]
index = ["whole population", "computer scientists"]

chocolate = pd.DataFrame(data=data, columns=columns, index=index)
print(chocolate)

                      Dark   Milk   White  Other
whole population     50000  60000  100000  35000
computer scientists    250    300     600    150


In [134]:
def compute_chi_squared(observed,expected):
    return(sum([((obs-exp)**2)/exp if exp!=0 else O for obs,exp in zip(observed,expected)]))

In [66]:
whole = [50000,60000,100000,35000]
scientists = [250,300,600,150]
N_whole = sum(whole)
N_scientists = sum(scientists)

expected = [N_scientists*prop/N_whole for prop in whole]
print(expected)
score = compute_chi_squared(scientists, expected)
print(f"score: {score}")

[265.3061224489796, 318.3673469387755, 530.6122448979592, 185.71428571428572]
score: 17.88461538461539


In [68]:
from scipy.stats import chi2, chisquare
print(f"critical value: {chi2.ppf(0.95, df=3)}")

# Associated p-value
pvalue = 1 - chi2.cdf(score, df=3)
print(f"associated p-value: {pvalue}")

critical value: 7.814727903251179
associated p-value: 0.0004646197573072719


We find a chi-square score higher than the critical value, so we reject H0 that computer scientists's tastes follow the national distribution of tastes.

In [51]:
chisquare(scientists,expected)

Power_divergenceResult(statistic=17.88461538461539, pvalue=0.0004646197573072709)

## Independence test

In [76]:
data = pd.read_csv("IBMdataset.csv")
data

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,No,Travel_Frequently,884,Research & Development,23,2,Medical,1,2061,...,3,80,1,17,3,3,5,2,0,3
1466,39,No,Travel_Rarely,613,Research & Development,6,1,Medical,1,2062,...,1,80,1,9,5,3,7,7,1,7
1467,27,No,Travel_Rarely,155,Research & Development,4,3,Life Sciences,1,2064,...,2,80,1,6,0,3,6,2,0,3
1468,49,No,Travel_Frequently,1023,Sales,2,3,Medical,1,2065,...,4,80,0,17,3,2,9,6,0,8


In [74]:
print(len(data)) #nb of employees
print(len(data.columns)) #nb of attributes 

1470
35


In [80]:
data.isna().any() #data.isna() prints whole table with T/F
# any() gives a recap by column

Age                         False
Attrition                   False
BusinessTravel              False
DailyRate                   False
Department                  False
DistanceFromHome            False
Education                   False
EducationField              False
EmployeeCount               False
EmployeeNumber              False
EnvironmentSatisfaction     False
Gender                      False
HourlyRate                  False
JobInvolvement              False
JobLevel                    False
JobRole                     False
JobSatisfaction             False
MaritalStatus               False
MonthlyIncome               False
MonthlyRate                 False
NumCompaniesWorked          False
Over18                      False
OverTime                    False
PercentSalaryHike           False
PerformanceRating           False
RelationshipSatisfaction    False
StandardHours               False
StockOptionLevel            False
TotalWorkingYears           False
TrainingTimesL

no missing value

In [82]:
data["Attrition"].describe()

count     1470
unique       2
top         No
freq      1233
Name: Attrition, dtype: object

In [88]:
data["JobSatisfaction"].describe()

count    1470.000000
mean        2.728571
std         1.102846
min         1.000000
25%         2.000000
50%         3.000000
75%         4.000000
max         4.000000
Name: JobSatisfaction, dtype: float64

In [91]:
crosstab = pd.crosstab(data.Attrition, data.JobSatisfaction, margins=True)
crosstab

JobSatisfaction,1,2,3,4,All
Attrition,,,,,
No,223,234,369,407,1233
Yes,66,46,73,52,237
All,289,280,442,459,1470


In [98]:
crosstab.iloc[[0]]

JobSatisfaction,1,2,3,4,All
Attrition,,,,,
No,223,234,369,407,1233


Remark: when considering multiple variables, global_df = product of local df. Here df = (4-1)\*(2-1)=3

In [115]:
crosstab.iloc[[1],4]

Attrition
Yes    237
Name: All, dtype: int64

In [116]:
no_ratio = 1233/1470
yes_ratio = 1 - no_ratio
expected_no = [crosstab.iloc[2,i]*no_ratio for i in range(5)]
print(expected_no)
expected_yes = [crosstab.iloc[2,i]*yes_ratio for i in range(5)]
print(expected_yes)

[242.40612244897957, 234.85714285714283, 370.73877551020405, 384.99795918367346, 1233.0]
[46.59387755102042, 45.14285714285715, 71.26122448979594, 74.00204081632656, 237.00000000000006]


In [140]:
print(crosstab.iloc[0].values, expected_no)
chi2_score = compute_chi_squared(crosstab.iloc[0].values, expected_no)
print(chi2_score)

chi2_score_yes = compute_chi_squared(crosstab.iloc[1].values, expected_yes)
print(chi2_score_yes)

print(chi2_score + chi2_score_yes)

[ 223  234  369  407 1233] [242.40612244897957, 234.85714285714283, 370.73877551020405, 384.99795918367346, 1233.0]
2.822247109831614
14.682829900516378
17.505077010347993


We must always compute the global chi2 score !

In [141]:
# Calculating the associated p-value
pvalue = 1 - chi2.cdf(chi2_score + chi2_score_yes, df=3)
print(pvalue)

0.000556300451038716


Rejection of H0

In [145]:
from scipy.stats.contingency import chi2_contingency

test_statistic, pvalue, dof, expctd =chi2_contingency(crosstab, correction=False)
print(test_statistic, pvalue)
print(expctd)

17.505077010347996 0.02525900441158742
[[ 242.40612245  234.85714286  370.73877551  384.99795918 1233.        ]
 [  46.59387755   45.14285714   71.26122449   74.00204082  237.        ]
 [ 289.          280.          442.          459.         1470.        ]]
